# Retrieve paintings and their descriptions from MET
This notebook retrieves European paintings and their descriptions from the Metropolitan Museum of Art from the period 13th-20th century.

### 0. Import libraries

In [ ]:
import os
import json
import requests
import polars as pl
from tqdm import tqdm
from PIL import Image
from io import BytesIO
from bs4 import BeautifulSoup

### 1. Load data

In [ ]:
DATA_PATH = "../../data/raw/"

In [ ]:
try:
    os.mkdir(DATA_PATH + "met_paintings/")
except FileExistsError:
    pass

In [ ]:
data = pl.read_csv(DATA_PATH + "met_objects.csv", ignore_errors=True)
data.head()

In [ ]:
european_paintings = data.filter(
    (pl.col("Department") == "European Paintings")
    & (pl.col("Is Public Domain") == True)
    & (pl.col("Object End Date") >= 1201)
    & (pl.col("Object End Date") < 2001)
)

european_paintings_details = european_paintings[
    ["Title", "Artist Display Name", "Object End Date", "Link Resource"]
].to_numpy()
european_paintings_details

In [ ]:
paintings_data = []

for painting_id, (title, artist, year, painting_url) in enumerate(tqdm(european_paintings_details)):
    response = requests.get(painting_url)
    response.raise_for_status()

    description = ""

    soup = BeautifulSoup(response.text, "html.parser")
    brief_description = (
        soup.find("div", {"class": "artwork__intro__desc js-artwork__intro__desc"})
        .get_text()
        .strip()
    )

    if len(brief_description) > 0:
        description += brief_description + "\n"

    try:
        catalog_entry = (
            soup.find("section", {"id": "catalogue-entry"})
            .find("div", class_="show-more__body js-show-more__body")
            .get_text()
            .strip()
        )
        description += catalog_entry
    except AttributeError:
        pass

    if len(description) > 0:
        image_url = soup.find("meta", {"property": "og:image"})["content"]
        image = requests.get(image_url).content

        paintings_data.append(
            {
                "id": painting_id,
                "title": title,
                "artist": artist,
                "year": year,
                "description": description,
            }
        )

        with open(DATA_PATH + "met_paintings/" + f"{painting_id}.png", "wb") as handler:
            handler.write(image)

        with open(DATA_PATH + "met_paintings_data.json", 'w') as json_file:
            json.dump(paintings_data, json_file, indent=4)  

In [ ]:
Image.open(BytesIO(image))

the first 730 were processed